About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





importing the dependencies

In [11]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [12]:
 import nltk
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
#printing stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

data preprocessing

In [15]:
#loading dataset to a pandas DataFrame
news_dataset=pd.read_csv('/train.csv')

In [16]:
news_dataset.shape

(20800, 5)

In [17]:
#print first five rows of dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [18]:
#counting number of missing values
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [19]:
#replacing null values with empty strings
news_dataset=news_dataset.fillna('')

In [20]:
 #merging author name and news title
 news_dataset['content']=news_dataset['author']+ '' +news_dataset['title']

In [21]:
print(news_dataset['content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [22]:
#separating data and label
X= news_dataset.drop(columns='label',axis=1)
Y=news_dataset['label']

In [23]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

stemming process of reducing a word to its root word
example actors,actress,acting==>act

In [37]:
port_stem=PorterStemmer()

In [45]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ', content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = '  '.join(stemmed_content)
    return stemmed_content



In [46]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [48]:
print(news_dataset['content'])

0        darrelllucushousedemaidewedidntevenseecomeysle...
1        danieljflynnflynnhillaryclintonbigwomanoncampu...
2               consortiumnewscomwhythetruthmightgetyoufir
3        jessicapurkisscivilianskilledinsingleusairstri...
4        howardportnoyiranianwomanjailedforfictionalunp...
                               ...                        
20795    jeromehudsonrappertitrumpaposterchildforwhites...
20796    benjaminhoffmannflplayoffsschedulematchupsando...
20797    michaeljdelamercedandrachelabramsmacysissaidto...
20798    alexansarynatorussiatoholdparallelexercisesinb...
20799                            davidswansonwhatkeepsthef
Name: content, Length: 20800, dtype: object


In [49]:
#separating data and label
X=news_dataset['content'].values
Y=news_dataset['label'].values

In [50]:
print(X)

['darrelllucushousedemaidewedidntevenseecomeysletteruntiljasonchaffetztweetedit'
 'danieljflynnflynnhillaryclintonbigwomanoncampusbreitbart'
 'consortiumnewscomwhythetruthmightgetyoufir' ...
 'michaeljdelamercedandrachelabramsmacysissaidtoreceivetakeoverapproachbyhudsonsbaythenewyorktim'
 'alexansarynatorussiatoholdparallelexercisesinbalkan'
 'davidswansonwhatkeepsthef']


In [51]:
print(Y)

[1 0 1 ... 0 1 1]


In [52]:
Y.shape

(20800,)

In [54]:
#converting textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(X)

X=vectorizer.transform(X)

In [55]:
print(X)

  (0, 5208)	1.0
  (1, 4896)	1.0
  (2, 4549)	1.0
  (3, 9741)	1.0
  (4, 7968)	1.0
  (5, 4994)	1.0
  (6, 11988)	1.0
  (7, 1010)	1.0
  (8, 6803)	1.0
  (9, 12883)	1.0
  (10, 85)	1.0
  (11, 4201)	1.0
  (12, 1165)	1.0
  (13, 9095)	1.0
  (14, 1698)	1.0
  (15, 8725)	1.0
  (16, 13016)	1.0
  (17, 17437)	1.0
  (18, 18102)	1.0
  (19, 9269)	1.0
  (20, 14162)	1.0
  (21, 9651)	1.0
  (22, 15084)	1.0
  (23, 12692)	1.0
  (24, 5799)	1.0
  :	:
  (20775, 1824)	1.0
  (20776, 8919)	1.0
  (20777, 408)	1.0
  (20778, 13973)	1.0
  (20779, 18501)	1.0
  (20780, 4165)	1.0
  (20781, 7747)	1.0
  (20782, 18134)	1.0
  (20783, 3726)	1.0
  (20784, 5551)	1.0
  (20785, 1573)	1.0
  (20786, 7464)	1.0
  (20787, 8238)	1.0
  (20788, 9851)	1.0
  (20789, 19648)	1.0
  (20790, 288)	1.0
  (20791, 4863)	1.0
  (20792, 10228)	1.0
  (20793, 16406)	1.0
  (20794, 11851)	1.0
  (20795, 9666)	1.0
  (20796, 2366)	1.0
  (20797, 13127)	1.0
  (20798, 890)	1.0
  (20799, 5474)	1.0


spliting dataset to training and test data

In [56]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.2,stratify=Y,random_state=2)

training model:logistic regression

In [57]:
model=LogisticRegression()

In [59]:
model.fit(X_train,Y_train)

LogisticRegression()

evaluation

accuracy score

In [60]:
#accuracy score on training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [61]:
print('accuracy score of training data :', training_data_accuracy)

accuracy score of training data : 0.9927283653846154


In [62]:
#accuracy score on test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [63]:
print('accuracy score of test data :', test_data_accuracy)

accuracy score of test data : 0.5329326923076924


making a predictive system


In [73]:
X_new=X_test[1]

prediction=model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [74]:
print(Y_test[1])

0
